# **Exploratory Data Analysis (EDA) of PHEE Data**

Four aims during the EDA of the PHEE Data.

1. Description of data attributes
3. Distribution of Text Length
4. Word Cloud of Text to understand most prevalent topics in the raw text
5. Standardize the terms in the columns to provide statistics on the patient populations experiencing potentially detrimental effects. Non-standardized terms will be used to extract events. 
  - Distribution of drugs
  - Distribution of patient demographics
  - Median time elapsed until a side-effect appeared in a 
  patient
  - Median duration a patient is on a medication
  - The route with the most severe side-effects in elderly patients is ...
  - The top 4 therapy areas (Treat-Disorder) in the dataset ...

In [14]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Load Data**

In [51]:
def pkl_load_dict(filename):
  """
    Load the dictionary data from a pickle file into a variable

    @P:
    filename (str): Name of the pkl file
    varname : Name of the df to save the pkl data

    @R:
    varname : Containing the pkl data
  """

  with open(filename, 'rb') as handle:
      df = pickle.load(handle)

  return df

df=pkl_load_dict('/content/drive/MyDrive/PHEE/output/data_df.pkl')

# **Description and Relationship of Select Medical/Data Terms**

**Subject** Patients involved in the medical event

Majority of the subjects are unknown (48%). The column should be preprocessed to group similar terms, like 'patient' and 'patients,' which may result in more meaningful data.

**Drug** A prescribed medication that causes a physiological effect 

The distribution of drugs in the dataset are uniform, with a slight majority of records with the drug 'methotrexate.' Unsure whether there is a mix of brand and generic drugs in the column.

**Adverse_event** Abbreviated as ADE. Denotes potentially harmful effects of medical therapies.

The most adverse event is 'induced.' Further grouping may be needed for the events.

**Effect** Indicates the outcome of the treatment.

There may be lots of overlap in terms, which may need to be normalized.

**Race** Indicates the subject’s race/nationality

Supporting the statistic that majority of the subject's are unknown, the majority of the race demographics are also unknown (92%). Standardization may be necessary.

**Age** Indicates the subject’s age

Supporting the statistic that majority of the subject's are unknown, the majority of the age demographics are also unknown (92%). Standardization of the ages is necessary. For example, '2-year-old child' and 'two' should be considered the same.

**Gender** Indicates the subject’s gender

Supporting the statistic that majority of the subject's are unknown, the majority of the age demographics are also unknown (82%). Standardization of the gender is necessary. For example, 'woman' and 'girl' should be considered the same.

**Text** Sentences extracted from biomedical literature, MEDLINE case reports, annotated with information
relevant to pharmacovigilance.


> **Ex** Diarrhoea, T-CD4+ lymphopenia and bilateral patchy pulmonary infiltrates developed in a male 60 yrs of age, who was treated with oxaliplatinum and 5-fluorouracil for unresectable rectum carcinoma.



In [15]:
def relative_freq(df,col):
  """
    Relective frequency of a column in a df

    @P:
      df (dataframe): Dataframe of data
      col (str): Name of the column of interest
    
  """
  return round(df[col].value_counts(normalize=True)*100,0).to_frame()

In [ ]:
print("An example sentence extracted from a MEDLINE case report: \n {} \n".format(df['Text'].iloc[5]))
relative_freq(df,'Treat-Disorder')

# **Distribution of Text Data Tokens**

**Distribution of Text Data Tokens** The text data contains text with a minimum number of tokens being 1 token and the maximum being 75 tokens. The median number of tokens is 17, while the average number of tokens is 18 tokens.

In [49]:
def dist_text_len(df_rw,output_pth):
  """
    Save the distribution of the text length as a '.png' file 
    to the output path

    @P: 
      df_rw (dataframe): Dataframe of raw data
      output_pth (string): Path to save the '.png' file
  """
  dict_txt_dist={'text_lst':[],'txt_len_lst':[]}
  dict_txt_dist['text_lst']=df_rw['Text'].tolist()
  
  for txt in dict_txt_dist['text_lst']:
    txt_lst=txt.split()
    dict_txt_dist['txt_len_lst'].append(len(txt_lst))

  df=pd.DataFrame.from_dict(dict_txt_dist)
  sns.distplot(dict_txt_dist['txt_len_lst'], kde=False, color='blue', bins=10)
  plt.savefig(output_pth+'/text_len_dist.png')

  return df.describe()


In [ ]:
dist_text_len(df,'/content/drive/MyDrive/PHEE/output')

# **Word Cloud of Text Data**